In [ ]:
import os
import tensorflow as tf

tces_file = 'astronet/tces-new+old.csv'
file_pattern = '/mnt/tess/astronet/tfrecords-11/*-*'
filenames = tf.io.gfile.glob(file_pattern)
    
filenames

In [ ]:
import pandas as pd

tce_table = pd.read_csv(tces_file, header=0)
tce_table = tce_table.drop(columns=['Disposition'])
tce_table.head(3)

In [ ]:
import numpy as np

print(np.unique(tce_table["flag_av"].fillna(''), return_counts=True))
print(np.unique(tce_table["flag_ch"].fillna(''), return_counts=True))
print(np.unique(tce_table["flag_yl"].fillna(''), return_counts=True))

In [ ]:
f = (tce_table["flag_yl"] != tce_table["flag_ch"]) & (~tce_table["flag_yl"].isna()) & (~tce_table["flag_ch"].isna())
tce_table[f]

In [ ]:
series = {}

for filename in filenames:
  tfr = tf.data.TFRecordDataset(filename)
  num_records = 0
  for record in tfr:
    num_records += 1
    ex = tf.train.Example.FromString(record.numpy())
    for k in ex.features.feature.keys():
      if k in ('global_view', 'local_view', 'secondary_view', 'toi_id'):
        continue
      f = ex.features.feature[k]
      if f.int64_list.value:
        v = f.int64_list.value[0]
      elif f.float_list.value:
        v = f.float_list.value[0]
      elif f.bytes_list.value:
        v = f.bytes_list.value[0].decode()
      else:
        continue

      if k not in series:
        series[k] = []
      series[k].append(v)
  print(filename, num_records)

In [ ]:
import pandas as pd

examples_table = pd.DataFrame.from_dict(series)

pd.set_option('display.max_columns', None)
examples_table.describe()

In [ ]:
from matplotlib import pyplot as plt

labels = ['E', 'N', 'J', 'S', 'B']
counts = [sum(examples_table['disp_{}'.format(l)] > 0) for l in labels]
ax = plt.bar(labels, counts)
for i in range(len(labels)):
    b = ax[i]
    height = b.get_height()
    x, y = b.get_xy()
    plt.annotate(
        '{:.0%}'.format(counts[i] / sum(counts)),
        (x + 0.1, y + height + 100))

In [ ]:
examples_table.describe()

In [ ]:
examples_table.head(5)

In [ ]:
print(np.unique(examples_table["flag_av"].fillna(''), return_counts=True))
print(np.unique(examples_table["flag_ch"].fillna(''), return_counts=True))
print(np.unique(examples_table["flag_yl"].fillna(''), return_counts=True))

In [ ]:
np.array(set(tce_table[tce_table['disp_E'] > 0]['tic_id'].values)
    - set(examples_table[examples_table['disp_E'] > 0]['tic_id'].values))

In [ ]:
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np

from astronet.data import preprocess


tess_data_dir = '/mnt/tess/lc'

def find_tce(tic_id, sector):
  with tf.device('cpu'):
    for filename in filenames:
      tfr = tf.data.TFRecordDataset(filename)
      for record in tfr:
        ex = tf.train.Example.FromString(record.numpy())
        if ((ex.features.feature["tic_id"].int64_list.value[0] == tic_id)
            and (ex.features.feature["Sectors"].int64_list.value[0] == sector)):
          print('TIC ID:', tic_id)
          print('E:', ex.features.feature["disp_E"].int64_list.value[0])
          print('N:', ex.features.feature["disp_N"].int64_list.value[0])
          print('J:', ex.features.feature["disp_J"].int64_list.value[0])
          print('S:', ex.features.feature["disp_S"].int64_list.value[0])
          print('Duration:', ex.features.feature["Duration"].float_list.value[0])
          return ex

    raise ValueError("{} not found in files: {}".format(tic_id, filenames))


def plot_tce(ex, label=''):
    global_view = np.array(ex.features.feature["global_view"].float_list.value)
    local_view = np.array(ex.features.feature["local_view"].float_list.value)
    secondary_view = np.array(ex.features.feature["secondary_view"].float_list.value)
    fig, axes = plt.subplots(1, 3, figsize=(20, 6), dpi=300)
    axes[0].plot(global_view, ".")
    axes[1].plot(local_view, ".", label=label)
    axes[2].plot(secondary_view, ".", label=label)
    plt.show()
    plt.close('all')


def plot_raw_lc(tic_id):
    time, flux = preprocess.read_and_process_light_curve(tic_id, tess_data_dir)
    fig = plt.figure(figsize=(15, 4), dpi=300)
    plt.plot(time, flux, '.', markersize=3)
    plt.show()
    time, flux = preprocess.read_and_process_light_curve(tic_id, tess_data_dir, 'SAP_FLUX')
    fig = plt.figure(figsize=(15, 4), dpi=300)
    plt.plot(time, flux, '.', markersize=3)
    plt.show()
    plt.close('all')


In [ ]:
import tensorflow as tf

from astronet import models
from astronet.ops import dataset_ops

config = models.get_model_config('AstroCNNModel', 'local_global_new')

ds = dataset_ops.build_dataset(
      file_pattern=file_pattern,
      input_config=config.inputs,
      batch_size=1,
      include_labels=False,
      reverse_time_series_prob=0,
      shuffle_filenames=True,
      shuffle_values_buffer=1000,
      repeat=1,
      use_tpu=False,
      one_hot_labels=(config.hparams.output_dim > 1),
      include_identifiers=True)

all_tics = []
for d in ds:
  def lam(e):
    if e.dtype == tf.int64:
        return e
    if tf.reduce_any(tf.math.is_nan(e)):
        tf.print(e, summarize=-1)
        raise ValueError('data has NaNs.')
    return e
  tic_id = d[1]['tic_id'].numpy().item()
  all_tics.append(tic_id)
  try:
    tf.nest.map_structure(lam, d)
  except ValueError as e:
    print(e)
    print(d[1])
    break
else:
  print('No NaNs found.')

if len(all_tics) == len(set(all_tics)):
  print('No duplicates found.')
else:
  print('Found duplicates!', len(all_tics) - len(set(all_tics)))

In [ ]:
ds = ds.cache()
def plot_ds_tce(ds, tic_id):
    for d in ds:
        if d[1]['tic_id'] == tic_id:
            for k, v in d[0].items():
                if k not in ['local_view', 'global_view', 'secondary_view']:
                    print(f'{k:25}: {v.numpy().item()}')
            global_view = np.array(d[0]['global_view'][0].numpy())
            local_view = np.array(d[0]['local_view'][0].numpy())
            secondary_view = np.array(d[0]['secondary_view'][0].numpy())
            fig, axes = plt.subplots(1, 3, figsize=(20, 6))
            axes[0].plot(global_view, '.-')
            axes[1].plot(local_view, '.-')
            axes[2].plot(secondary_view, '.-')
            plt.show()
            plt.close('all')
            return

In [ ]:
tic_id = 424985843
plot_ds_tce(ds, tic_id)
plot_raw_lc(tic_id)
tce_table[tce_table['tic_id'] == tic_id]